# Nonlinear analysis -Assignment 2

04.10.2023 - Jérémie Engler and Laure Toullier

In [118]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('bmh')
import math

np.set_printoptions(precision=3, suppress=True)

Params = plt.rcParams
Params['figure.figsize'] = (14, 7) 

### Part b

#### Geometry

In [119]:
#Geometric data for the truss:
n=3  #nb of nodes

support=[0,2] #number of the nodes where there is a support
act_P=[1]
rests=[1,1,0,0,1,1] #restraints 
DOF=2*n #degree of freedom

a=3000 #mm
b=4000 #mm

In [120]:
#Material Stiffness
E = 200e3  # N/mm^2
sigma_0=500e6 #Pa
eps_0=sigma_0/E
alpha=0.02

In [121]:
#Load definition
load=8675*10**3 #N, value of the applied load
P=np.zeros(DOF).reshape(n,2) 
theta_P=60 #°, inclination of the load regarding the x-axis

    #Application of the load at the corresponding nodes:

P[1]=[load*np.cos(np.radians(theta_P)),load*np.sin(np.radians(theta_P))]
print(P)

[[      0.          0.   ]
 [4337500.    7512770.378]
 [      0.          0.   ]]


In [122]:
#Generate the nodes of the truss and their location (if 1st node at (0,0))

nodes=np.zeros(n*2).reshape(n,2)

for i in range(n):
    nodes[i,0]=a*i
    if i%2==0:
        nodes[i,1]=0
    else:
        nodes[i,1]=b

print(nodes)

[[   0.    0.]
 [3000. 4000.]
 [6000.    0.]]


In [123]:
#Connectivity: connect nodes with bar element

connectivity=[]

for i in range(n):
    if i<(n-2):
        connectivity.append([i+1, i+2])
#        connectivity.append([i+1, i+3])
    if i==n-2:
        connectivity.append([i+1, i+2])
        
Connectivity=np.array(connectivity)

nb_members=int(np.size(Connectivity)/2)
    
print(Connectivity)

[[1 2]
 [2 3]]


In [124]:
# Area: area of each bar of the truss

#A=10*10**3 #mm²     #members area equal in the entire truss

#area=np.zeros(nb_members)

#for i in range(np.size(area)):
#    area[i]=A
    
area=np.array([15000, 2000])

#### Stiffness

In [125]:
def stiffness (E, area, nodes, nb_members, DOF, Connectivity):


    # Unit stiffness matrix of bar in local coordinate system
    K_unit = np.array([[1, 0, -1, 0], 
                   [0, 0,  0, 0], 
                   [-1, 0, 1, 0], 
                   [0, 0, 0, 0]])
    
    #Angle and length of each bar
    angles=[]
    length=[]
    

    for i in range(nb_members):
        
        x1=nodes[Connectivity[i,0]-1,0]      #For each node at the extremity of each bar, we get its location in x and y.
        y1=nodes[Connectivity[i,0]-1,1]
        x2=nodes[Connectivity[i,1]-1,0]
        y2=nodes[Connectivity[i,1]-1,1]
        dx=x2-x1
        dy=y2-y1
        length.append(np.sqrt(dx**2+dy**2))
        
        if np.isclose(y1, y2):
            if np.isclose(x1, x2):
                angles.append(90)
            else:
                angles.append(0)
        else:
            theta=np.arcsin((y2-y1)/np.sqrt(dx**2+dy**2)) #calculate the angle between the x-axis and the bar element
            angles.append(math.degrees(theta))
    
    #local stiffness matrix for each bar
    k_local=[]
    
    for i in range(nb_members):
        k_local.append(E*area[i]/length[i])

    #Transformation matrix for each bar element of the truss
    c=[]
    s=[]
    for i in range(nb_members):
        c.append(np.cos(np.radians(angles[i])))
        s.append(np.sin(np.radians(angles[i])))
    
    T=np.zeros((nb_members,4,4))

    for i in range(nb_members):
        T[i,0,0]=c[i]
        T[i,0,1]=s[i]
        T[i,1,0]=-s[i]
        T[i,1,1]=c[i]
        T[i,2,2]=c[i]
        T[i,2,3]=s[i]
        T[i,3,2]=-s[i]
        T[i,3,3]=c[i]
        
    #Stiffness for each bar in global coordinates system
    k_global=[]

    for i in range(nb_members):
        k=k_local[i]*T[i].T@K_unit@T[i] 
        k_global.append(k)

    #Stiffness in global system
    K_globalCS_bar=np.zeros((nb_members, DOF,DOF))
    K_global=np.zeros((DOF,DOF))
    k_global = np.array(k_global)

    for i, c in enumerate(Connectivity):       
    #we build all the global stiffness matrix for each bar in the entire system and sum them.
        u, v = c
        K_global[2*u-2:2*u,2*u-2:2*u] += k_global[i, 0:2, 0:2]
        K_global[2*u-2:2*u,2*v-2:2*v] += k_global[i, 0:2, 2:4]
        K_global[2*v-2:2*v,2*u-2:2*u] += k_global[i, 2:4, 0:2]
        K_global[2*v-2:2*v,2*v-2:2*v] += k_global[i, 2:4, 2:4]
    
    #global stiffness matrix for each bar in the entire system 
        K_globalCS_bar[i, 2*u-2:2*u,2*u-2:2*u] = k_global[i, 0:2, 0:2]
        K_globalCS_bar[i, 2*u-2:2*u,2*v-2:2*v] = k_global[i, 0:2, 2:4]
        K_globalCS_bar[i, 2*v-2:2*v,2*u-2:2*u] = k_global[i, 2:4, 0:2]
        K_globalCS_bar[i, 2*v-2:2*v,2*v-2:2*v] = k_global[i, 2:4, 2:4]
        
    return (k_local,K_global, K_globalCS_bar, T, length)

#### Equilibrium of the entire system : K*U=R

In [126]:
#Calculate the displacements of the nodes with a load applied

def displacement(support, act_P, P, K_global, DOF):
    
    F_red=np.zeros(DOF-2*len(support))

    for i in range(n):
        if i in act_P:
            F_red[i*2-2]=P[i,0]
            F_red[i*2-1]=P[i,1]  
        
    a=2*(n-len(support)) 
    K_red=np.zeros((a,a)) #reduced stiffness matrix for the nodes where there are displacements
    K_red=K_global[2:a+2, 2:a+2]

    u_red=np.linalg.inv(K_red)@F_red #reduced vector displacements for the nodes concerned by displacements

    u_vec=np.zeros(n*2) #global vector for displacements
    u_vec[2:n*2-2]=u_red[:]

    return(u_vec) 


In [127]:
#Support reactions

def support_reactions (K_global, u_vec):
    F = K_global @ u_vec #N
    return(F)

In [128]:
#Axial forces in each bar of the truss

def axial_load_0(nb_members, K_globalCS_bar,u_vec):

    F_bar=[]
    F_bar_reduced=np.zeros((nb_members,4))
    F_local_CS=[]
    axial_load=[]

    for i in range(nb_members):
        F_bar.append(K_globalCS_bar[i,:,:]@u_vec)   #forces applied at each extermity of each bar in global coordinates in global system
    F_bar=np.array(F_bar)

    for i in range(nb_members): #forces applied at the extremity of each bar in global coordinates in local system
        index_1=Connectivity[i,0] #value of 1st node of the bar
        index_2=Connectivity[i,1] #value of 2nd node of the bar
        F_bar_reduced[i,0]=F_bar[i,(index_1-1)*2] 
        F_bar_reduced[i,1]=F_bar[i,(index_1)*2-1]
        F_bar_reduced[i,2]=F_bar[i,(index_2-1)*2]
        F_bar_reduced[i,3]=F_bar[i,index_2*2-1]     

    for i in range(nb_members): #forces applied at the extremity of each bar in local coordinates in local system
        F_local_CS.append(T[i]@F_bar_reduced[i])   
    F_local_CS=np.array(F_local_CS)

    for i in range(nb_members): #axial load in each bar in N
        axial_load.append(F_local_CS[i,2])
    
    axial_load=np.array(axial_load)
    return axial_load

def axial_load_1(K_globalCS_bar, Connectivity, u_vec, T, nb_members):
    axialF = []
    for i in range(nb_members):
        F_elei = K_globalCS_bar[i,:,:] @ u_vec
        index_red = 2*Connectivity[i]
        F_elei_red = F_elei[[index_red[0]-2, index_red[0]-1, index_red[1]-2, index_red[1]-1]]
        F_elei_local = T[i] @ F_elei_red
        Fi = F_elei_local[2]
        axialF.append(Fi)
    return axialF


#### Newton Raphson Method

In [129]:
#Initialisation

u_0=np.array([0,0,0,0,0,0]).reshape((3,2)) #initial nodal displacement
k, K_global, K_globalCS_bar, T, L=stiffness(E, area, nodes, nb_members, DOF,Connectivity) #initial stiffness matrix
load_incr=2 #increment for loadstep
iter_lim=20
tol=area[0]*sigma_0/1e5

In [141]:
 #strains

def strain(Connectivity, area, E, u, L, T, nb_members):

    u_red=np.zeros(nb_members*4).reshape((nb_members,4))
    
    for i in range(nb_members):  
        u_red[i, 0]=u[Connectivity[i,0]-1,0]      
        u_red[i, 1]=u[Connectivity[i,0]-1,1]
        u_red[i, 2]=u[Connectivity[i,1]-1,0]
        u_red[i, 3]=u[Connectivity[i,1]-1,1]
    
    print(u_red)
    
    u_bar=[]
    for i in range(nb_members):
        u_bar.append(T[i]@u_red[i,:])
    print(u_bar)
    strain=[]
    for i in range(nb_members):  
        strain.append((u_bar[i,2]-u_bar[i,0])/L[i])
    
    print(strain)
        
    return strain

In [142]:
#internal force for each displacement

def internal_force(Connectivity, area, E, u, L, T, sigma_0, eps_0, nb_members, DOF, rests):
    
    Pr=[]
    strains=strain(Connectivity, area, E, u, L, T, nb_members)
    for i in range(nb_members):
        a= area[i]*sigma_0/2*((3*strains[i]/eps_0)-(strains[i]/eps_0)**3)
        Pr.append(a)
    Pr=np.array(Pr) 
    
    return Pr

In [143]:
P_applied=P.reshape((n,2))/load_incr
delta_u=displacement(support, act_P, P_applied, K_global, DOF)
delta_u=delta_u.reshape((n,2))
u=u_0+delta_u
print(P_applied)
nodes=nodes+np.array(delta_u).reshape(3,2) #new global displacements 

Pr=internal_force(Connectivity, area, E, u, L, T, sigma_0, eps_0, nb_members, DOF, rests)
print(Pr)
P=support_reactions (K_global, u.reshape(DOF,1))
P_bar=np.array([np.sqrt(P[0]**2+P[1]**2), np.sqrt(P[4]**2+P[5]**2) ]).reshape((1,2)).flatten()
print(P_bar)

R=[]
for i in range(nb_members):
    R.append(P_bar[i]-Pr[i])


print(R)

[[      0.          0.   ]
 [2168750.    3756385.189]
 [      0.          0.   ]]
[[0.    0.    0.141 8.55 ]
 [0.141 8.55  0.    0.   ]]
[array([0.   , 0.   , 6.925, 5.017]), array([-6.756,  5.243,  0.   ,  0.   ])]


TypeError: list indices must be integers or slices, not tuple

In [144]:
#essai de généralisation....

steps = 1;
lams =  np.linspace(1,load_incr)/load_incr


for lam_i in range(np.size(lams)):
    lam = lams[lam_i];
    for i in range(iter_lim):
        P = lam * P
        #displacement
        delta_u=displacement(support, act_P, P_applied, K_global, DOF)
        delta_u=delta_u.reshape((n,2))
        u=u_0+delta_u
        nodes=nodes+np.array(delta_u).reshape(3,2) #new global displacements 
        
        Pr = internal_force(Connectivity, area, E, u, L, T, sigma_0, eps_0, nb_members, DOF, rests)
        P=support_reactions (K_global, u.reshape(DOF,1)) #attention, je ne suis pas sûre de ça !!!
        P_bar=np.array([np.sqrt(P[0]**2+P[1]**2), np.sqrt(P[4]**2+P[5]**2) ]).reshape((1,2)).flatten() #attention, je ne suis pas sûre de ça !!!
        
        R=[]
        for i in range(nb_members):
            R.append(P_bar[i]-Pr[i])
        
        k, K_global, K_globalCS_bar, T, L=stiffness(E, area, nodes, nb_members, DOF,Connectivity)
        
        for i in range(nb_members):
            delta_u=displacement(support, act_P, P_applied, K_global, DOF)
            #du = np.linalg.inv(K_globalCS_bar[i,:,:]) * R[i]    #du code de la prof
            u = u + delta_u
            
 #       for m in range(nb_members):
  #          strains[m] = strain(Connectivity, area, E, u, L, T, nb_members)
  #          if abs(strains(m)) >= eps_0
   #             elems(m, 4) = alpha * E





[[0.    0.    0.141 8.55 ]
 [0.141 8.55  0.    0.   ]]
[array([0.   , 0.   , 6.925, 5.017]), array([-6.756,  5.243,  0.   ,  0.   ])]


TypeError: list indices must be integers or slices, not tuple

In [15]:
#Conclusion - Results - Support reactions and axial loads

nodes_x=[]
nodes_y=[]

for i in range(n):
    nodes_x.append(nodes[i,0])
    nodes_y.append(nodes[i,1])
    

x=np.linspace(0,18000,3000)

nodes_xx=[]
nodes_yy=[]

for u, v in Connectivity:
    x1=nodes[u-1,0]
    x2=nodes[v-1,0]
    y1=nodes[u-1,1]
    y2=nodes[v-1,1]
    nodes_xx.append([x1,x2])
    nodes_yy.append([y1,y2])
nodes_xx = np.array(nodes_xx)
nodes_yy = np.array(nodes_yy)


# Displacements
u_vec_array=u_vec.reshape((n,2))
f = 100
nodes_displ = nodes + u_vec_array #list of displaced nodes
nodes_displ_exaggerated = nodes + u_vec_array * f

nodes_displ_x=[]
nodes_displ_y=[]
for dispx, dispy in nodes_displ:
    nodes_displ_x.append(dispx)
    nodes_displ_y.append(dispy)

nodes_displ_x_ex=[]
nodes_displ_y_ex=[]
for u, v in Connectivity:
    x1=nodes_displ_exaggerated[u-1,0]
    x2=nodes_displ_exaggerated[v-1,0]
    y1=nodes_displ_exaggerated[u-1,1]
    y2=nodes_displ_exaggerated[v-1,1]
    nodes_displ_x_ex.append([x1,x2])
    nodes_displ_y_ex.append([y1,y2])

print(nodes_xx)

compression_label=False
tension_label=False
displaced_label=False
for nx, ny, dx, dy, axload in zip(nodes_xx, nodes_yy, nodes_displ_x_ex, nodes_displ_y_ex, axial_load):

    # Graph legend
    if axload > 0:
        color='red'
        label='tension' if tension_label is False else None
        tension_label=True
    else:
        color='blue'
        label='compression' if compression_label is False else None
        compression_label=True

    plt.text(
        x=nx.mean(),
        y=ny.mean(),
        s=f"N={axload:+.0f} N",  # :.2f veut dire deux chiffres après la virgule
        bbox=dict(facecolor='white', edgecolor=color, boxstyle='round, pad=1') # optionnel, elève ça si tu veux
    )
    
    # plot the bars
    plt.plot(nx, ny, color=color, linewidth=2, label=label)

    # displaced bars
    plt.plot(dx, dy, '-.', color='k', linewidth=1, label='displaced truss' if displaced_label is False else None)
    displaced_label = True
    plt.title(f"Displacements are exaggerated by a factor of {f:.0f}")

# Forces applied on the truss and support reactions
force_label = False
for (nx, ny), fx, fy in zip(nodes, F[::2], F[1::2]):
    facteur = 0.01
    if not (np.isclose(fx, 0.0) and np.isclose(fy, 0.0)):
        plt.arrow(
            x=nx - fx*facteur,
            y=ny - fy*facteur,
            dx=fx*facteur,
            dy=fy*facteur,
            color='green', width=100,
            length_includes_head=True,
            label='Loads in N' if force_label is False else None
        )
        force_label = True
        
        #text for loads and support reactions values
        plt.text(
            x=nx - 0.5*fx*facteur,
            y=ny - 0.5*fy*facteur,
            s=f"Fx={fx:.0f} N\nFy={fy:.0f} N",  # :.0f no decimal
            bbox=dict(facecolor='white', edgecolor='g', boxstyle='round, pad=1') 
        )


# Displaced nodes
plt.plot(*nodes_displ_exaggerated.T, 'x', markersize=10, label='displaced nodes', color='Green')  
# Nodes not displaced
plt.plot(*nodes.T, 'o', mfc='w', mew=2, mec='k')
plt.tight_layout()

plt.legend()

NameError: name 'u_vec' is not defined

In [ ]:
#Conclusion - Results - Displacement


compression_label=False
tension_label=False
displaced_label=False
for nx, ny, dx, dy, axload in zip(nodes_xx, nodes_yy, nodes_displ_x_ex, nodes_displ_y_ex, axial_load):

    # Graph legend
    if axload > 0:
        color='red'
        label='tension' if tension_label is False else None
        tension_label=True
    else:
        color='blue'
        label='compression' if compression_label is False else None
        compression_label=True

    # plot the bars
    plt.plot(nx, ny, color=color, linewidth=2, label=label)

    # displaced bars
    plt.plot(dx, dy, '-.', color='k', linewidth=1, label='displaced truss' if displaced_label is False else None)
    displaced_label = True
    plt.title(f"Displacements are exaggerated by a factor of {f:.0f}")

# Forces applied on the truss and support reactions

force_label = False
for (nx, ny), fx, fy, nx, ny, dx, dy in zip(nodes, F[::2], F[1::2],
                                    nodes_displ.flatten()[::2], nodes_displ.flatten()[1::2],
                                    u_vec[::2], u_vec[1::2]):
    facteur = 0.01
    if not (np.isclose(fx, 0.0) and np.isclose(fy, 0.0)):
        plt.arrow(
            x=nx - fx*facteur,
            y=ny - fy*facteur,
            dx=fx*facteur,
            dy=fy*facteur,
            color='green', width=100,
            length_includes_head=True,
            label='Loads in N' if force_label is False else None
        )
        force_label = True

    #text for displaced nodes (dx,dy)
    plt.text(
        x=nx,
        y=ny,
        s=f"dx={dx:+.3f} mm\ndy={dy:+.3f} mm",  
        bbox=dict(facecolor='white', edgecolor=color, boxstyle='round, pad=1') 
    )

# Displaced nodes
plt.plot(*nodes_displ_exaggerated.T, 'x', markersize=10, label='displaced nodes', color='Green')  
# Nodes not displaced
plt.plot(*nodes.T, 'o', mfc='w', mew=2, mec='k')
plt.tight_layout()

plt.legend()